In [143]:
# Setting up libraries
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import sklearn as skl
import ast

from config import user, password, hostname

# Preprocessing

### Connecting to database

In [109]:
# Create engine
engine = create_engine(f'postgres://{user}:{password}@{hostname}/twitter_vs_stocks')

# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['stock', 'tweets_text', 'twitter_vs_stocks']

In [127]:
# Create dataframe from SQL table
twitter_vs_stocks = pd.read_sql_table(
    'twitter_vs_stocks',
    con=engine)
twitter_vs_stocks.set_index(['date'], inplace=True)
twitter_vs_stocks.rename({'volume': 'volume_traded'}, axis=1, inplace=True)
twitter_vs_stocks.head()

,tokenized_text,like_count,quote_count,reply_count,retweet_count,change,volume_traded
date,,,,,,,
2021-07-09,"['solar', 'powerwall', 'battery', 'ensures', '...",37454,400,4443,2871,4.140015,18118500
2021-07-09,"['tesla', 'solar', 'roof', 'powerwall', 'major...",13972,119,2185,1407,4.140015,18118500
2021-07-09,"['autonomous', 'spacex', 'droneship', 'shortfa...",63291,860,3653,6553,4.140015,18118500
2021-07-09,"['electrekco', 'bought', 'first', 'tesla', 'he...",0,0,0,979,4.140015,18118500
2021-07-08,"['maybe', 'movie', 'gaslit', 'us']",26485,92,1927,1045,8.159973,22773300


### Defining the most common words in tweets

In [128]:
# define function for counting words
def count_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq = sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]

In [129]:
# count words in tweets
words_in_tweets = count_words(twitter_vs_stocks.tokenized_text)
words_in_tweets[:20]

[('spacex', 117),
 ('tesla', 81),
 ('falcon', 43),
 ('launch', 40),
 ('first', 34),
 ('dragon', 27),
 ('model', 22),
 ('mission', 21),
 ('starship', 20),
 ('crew', 20),
 ('flight', 18),
 ('landing', 17),
 ('space_station', 17),
 ('starlink', 16),
 ('doo', 15),
 ('stage', 15),
 ('next', 14),
 ('giga', 14),
 ('nasa', 14),
 ('doge', 13)]

In [130]:
# analyse only words that appear 3 or more times
words_to_analyse = [word for word, freq in words_in_tweets if freq >= 3]
len(words_to_analyse)

330

In [131]:
words_to_analyse[:10]

['spacex',
 'tesla',
 'falcon',
 'launch',
 'first',
 'dragon',
 'model',
 'mission',
 'starship',
 'crew']

### Count words in tweets

In [132]:
# convert text tows from string to list 
twitter_vs_stocks.tokenized_text = [ast.literal_eval(row) for row in twitter_vs_stocks.tokenized_text]

In [133]:
# count words in the tokenized text (tweets)
for column in words_to_analyse:
    count_list = []
    for row in np.arange(0,len(twitter_vs_stocks.tokenized_text)):
         count_list.append(twitter_vs_stocks.tokenized_text[row].count(column))
    twitter_vs_stocks[column] = count_list

In [134]:
twitter_vs_stocks.head()

,tokenized_text,like_count,quote_count,reply_count,retweet_count,change,volume_traded,spacex,tesla,falcon,...,tonight,volume,music,cell,saocom,government,astro_doug,astrobehnken,special,interests
date,,,,,,,,,,,,,,,,,,,,,
2021-07-09,"[solar, powerwall, battery, ensures, home, nev...",37454,400,4443,2871,4.140015,18118500,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-09,"[tesla, solar, roof, powerwall, major, new, ho...",13972,119,2185,1407,4.140015,18118500,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-09,"[autonomous, spacex, droneship, shortfall, gra...",63291,860,3653,6553,4.140015,18118500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-09,"[electrekco, bought, first, tesla, herewhat, h...",0,0,0,979,4.140015,18118500,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-08,"[maybe, movie, gaslit, us]",26485,92,1927,1045,8.159973,22773300,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Scaling and Features Extraction

In [136]:
X = twitter_vs_stocks.drop(columns=['tokenized_text', 'change', 'volume_traded'])
y_change = twitter_vs_stocks.change
y_volume = twitter_vs_stocks.volume

#### Tagret is Change

In [137]:
# Split dataset to training and testing
X_train, X_test, y_change_train, y_change_test = train_test_split(X, y_change, random_state=12)

In [140]:
# Create scaler instance
scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [142]:
X_train_scaled

array([[-0.85407441, -0.49206023, -0.57366483, ..., -0.04816831,
        -0.04816831, -0.06819943],
       [ 0.47466098,  0.15989215, -0.08196724, ..., -0.04816831,
        -0.04816831, -0.06819943],
       [ 0.08585003, -0.3510401 , -0.09456689, ..., -0.04816831,
        -0.04816831, -0.06819943],
       ...,
       [ 1.10598082,  0.46177445,  0.30807759, ..., -0.04816831,
        -0.04816831, -0.06819943],
       [-0.10359992, -0.37655128, -0.26201784, ..., -0.04816831,
        -0.04816831, -0.06819943],
       [-0.49784035, -0.30899389, -0.38863658, ..., -0.04816831,
        -0.04816831, -0.06819943]])

In [147]:
# Reduce the number of components in X to 10 using PCA
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)

#### Tagret is Volume Traded

In [148]:
# Split dataset to training and testing
X_train, X_test, y_volume_train, y_volume_test = train_test_split(X, y_volume, random_state=10)

In [149]:
scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [150]:
# Reduce the number of components in X to 10 using PCA
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)

# ML Model